# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [174]:
# !pip install scipy
# !pip install Pillow
# !pip install imageio
# !pip install tensorflow
# !pip install matplotlib
#!pip install opencv-python

In [175]:
import numpy as np
import os
import imageio
import datetime
import os
import cv2
from scipy import misc

We set the random seed so that the results don't vary drastically.

In [176]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.compat.v1.random.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [177]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())

batch_size = 30 #experiment with the batch size

## Visualization

Let us plot some sample images and run some transformations on the image so see the impact.

In [178]:
# ## Pick some random sequence
import matplotlib.pyplot as plt

data_dir_train = 'Project_data/train/'
random_sequence = train_doc[np.random.randint(len(train_doc))].strip().split(';')[0]+'/'
images = os.listdir(data_dir_train + random_sequence)
# Create a code to visualize one instance of all the 30 images present in the sequence
plt.figure(figsize=(10, 10))
i = 0
random_images = []
for img in images:
  ax = plt.subplot(6, 5, i + 1)
  i = i + 1
  random_images.append(os.path.join(data_dir_train, random_sequence, img))
  img_bgr = cv2.imread(os.path.join(data_dir_train, random_sequence, img))
  img_mp = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB) # This is needed because opencv uses BGR convention and matplotlib uses RGB
  imgplot= plt.imshow(img_mp)
  plt.axis("off")
  plt.plot()

In [179]:
# Random Image - Original
random_image = random_images[np.random.randint(30)]
image_bgr = cv2.imread(random_image)
plt.imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
print(image_bgr.shape)

In [180]:
## Cropped Image Sample
crop_img = image_bgr[10:120, 10:160]
plt.imshow(cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB))
print(crop_img.shape)

In [181]:
# Resize Image Sample
dim = (120, 120)
resized_img = cv2.resize(crop_img, dim, interpolation = cv2.INTER_AREA)
plt.imshow(cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB))
print(resized_img.shape)

In [182]:
# Normalized Image
plt.imshow(cv2.cvtColor(cv2.normalize(resized_img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F), cv2.COLOR_BGR2RGB))

## Generator

This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [184]:
def crop_image(image, size = 10):
     # cropping the image
    image_x = image.shape[0]
    image_y = image.shape[1]
    return image[size:image_x, size:image_y]

def normalize_minmax(image_resized, batch_data, folder, idx):
    batch_data[folder,idx,:,:,0] = cv2.normalize(image_resized[:,:,0], None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    batch_data[folder,idx,:,:,1] = cv2.normalize(image_resized[:,:,1], None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    batch_data[folder,idx,:,:,2] = cv2.normalize(image_resized[:,:,2], None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    return batch_data   

def crop_and_normalize(image,batch_data, folder, idx, y=120,z=120):
    
    #crop the images and resize them. Note that the images are of 2 different shape 
    #and the conv3D will throw error if the inputs in a batch have different shapes
    image_cropped = crop_image(image)
    image_resized = cv2.resize(image_cropped, (y,z), interpolation = cv2.INTER_AREA)
                    
    # using min max normalization.
    pending_batch_data = normalize_minmax(image_resized, batch_data, folder, idx)
    return pending_batch_data

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    x = 30 # taking 5 images from each video
    img_idx = range(1,x) #create a list of image numbers you want to use for a particular video
    y = 120 # image dim
    z = 120 # image dim
    while True:
        t = np.random.permutation(folder_list)
        num_batches =  len(t) // batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size, 5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                
                ## get images from the folder
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    batch_data = crop_and_normalize(image, batch_data, folder, idx)
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                
            yield batch_data, batch_labels 
            
        # The length of the folder list could leave some residue folders, the below code deals with it
        # Number of pending batches
        pending_batches = len(t) % batch_size
        
        pending_batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        pending_batch_labels = np.zeros((batch_size, 5)) # batch_labels is the one hot representation of the output
        
        for folder in range(pending_batches): # iterate over the batch_size

            # getimages from the folder
            imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                    
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    
                image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                # using min max normalization.
                pending_batch_data = crop_and_normalize(image, pending_batch_data, folder, idx)
                    
            pending_batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            
        yield pending_batch_data, pending_batch_labels
            

# g = generator(data_dir_train, train_doc, 30)
# next(g)

In [185]:
# g = generator(data_dir_train, train_doc, 30)
# p = 0
# for k in g:
#     p+=1
# print(p)

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [186]:
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 1 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 1


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [187]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

# Building a 3 D Convolution model.

model = Sequential()

model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(30,120,120,3)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [188]:
optimiser =  'sgd' #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 28, 118, 118, 32)  2624      
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 14, 59, 59, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 12, 57, 57, 64)    55360     
_________________________________________________________________
max_pooling3d_10 (MaxPooling (None, 6, 28, 28, 64)     0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 301056)            0         
_________________________________________________________________
dense_14 (Dense)             (None, 256)               77070592  
_________________________________________________________________
dense_15 (Dense)             (None, 256)              

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [195]:
batch_size = 30
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

## Callbacks

Here we define few callbacks which we will use later when we fit the complete model.

In [190]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

#LR = # write the REducelronplateau code here
callbacks_list = []

W0423 18:34:33.455706 140388283520832 callbacks.py:1071] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.


The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [196]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print('steps_per_epoch: {}, validation_steps: {}'.format(steps_per_epoch, validation_steps))

steps_per_epoch: 23, validation_steps: 4


## Ablation Experiment

Let us now fit the model with 1 epoch just to validate if the model is working.
Notice I have not added any callbacks. 

In [192]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=[], validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 30
23/23 [==============================] - 61s 3s/step - loss: 2.6429 - categorical_accuracy: 0.2101 - val_loss: 1.2796 - val_categorical_accuracy: 0.1750


The model seems to be working well, there are no errors. 

## Overfitting on training data. 

Let us overfit on the training data to see if the model is able to learn from the data. 

In [194]:
batch_size = 30
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [197]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print('steps_per_epoch: {}, validation_steps: {}'.format(steps_per_epoch, validation_steps))

steps_per_epoch: 23, validation_steps: 4


In [198]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=10, verbose=1, 
                    callbacks=[], validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 30
Epoch 1/10
23/23 [==============================] - 61s 3s/step - loss: 1.3581 - categorical_accuracy: 0.3870 - val_loss: 1.2029 - val_categorical_accuracy: 0.2833
Epoch 2/10
23/23 [==============================] - 61s 3s/step - loss: 1.1970 - categorical_accuracy: 0.4754 - val_loss: 0.9375 - val_categorical_accuracy: 0.4250
Epoch 3/10
23/23 [==============================] - 61s 3s/step - loss: 1.1161 - categorical_accuracy: 0.5333 - val_loss: 1.2791 - val_categorical_accuracy: 0.3667
Epoch 4/10
23/23 [==============================] - 59s 3s/step - loss: 0.9894 - categorical_accuracy: 0.5797 - val_loss: 0.8876 - val_categorical_accuracy: 0.4667
Epoch 5/10
23/23 [==============================] - 60s 3s/step - loss: 0.6823 - categorical_accuracy: 0.7130 - val_loss: 0.9129 - val_categorical_accuracy: 0.4750
Epoch 6/10
23/23 [==============================] - 60s 3s/step - loss: 0.5775 - categorical_accuracy: 0.7638 - val_loss: 0.7886